# collabarative filtering

importing required libraries

In [22]:
import numpy as n
import pandas as p
import matplotlib.pyplot as plt


In [ ]:
!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

getting data from csv file

In [23]:
movies_df=p.read_csv('movies.csv')
rating_df=p.read_csv('ratings.csv')


In [24]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


getting year separate form title

In [25]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

C:\Users\hemas\AppData\Local\Temp\ipykernel_5856\3106782265.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [26]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


drop genre column from movies_df 

In [27]:
movies_df=movies_df.drop(['genres'],axis=1)
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


rating_df

In [28]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


drop timestamp column from rating_df

In [29]:
rating_df=rating_df.drop(['timestamp'],axis=1)
rating_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


user - based filtering

In [30]:
userinput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputmovies = p.DataFrame(userinput)
inputmovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


adding movieid to inputmovies

In [31]:
inputid=movies_df[movies_df['title'].isin(inputmovies['title']).tolist()]
inputid

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
293,296,Pulp Fiction,1994
1246,1274,Akira,1988
1885,1968,"Breakfast Club, The",1985


merging  inputid and inputmovies

In [32]:
inputmovies=p.merge(inputid,inputmovies)
inputmovies

,movieId,title,year,rating
0,1,Toy Story,1995,3.5
1,2,Jumanji,1995,2.0
2,296,Pulp Fiction,1994,5.0
3,1274,Akira,1988,4.5
4,1968,"Breakfast Club, The",1985,5.0


getting users who watched movies in inputmovies

In [33]:
usersubset=rating_df[rating_df['movieId'].isin(inputmovies['movieId'].tolist())]
usersubset.shape,usersubset.head()

((196623, 3),
      userId  movieId  rating
 19        4      296     4.0
 441      12     1968     3.0
 479      13        2     2.0
 531      13     1274     5.0
 681      14      296     2.0)

groupby userid

In [34]:
type(usersubset)
usersubsetgroup = usersubset.groupby(usersubset['userId'])
usersubsetgroup

In [35]:
usersubsetgroup.get_group(13)

,userId,movieId,rating
479,13,2,2.0
531,13,1274,5.0


In [36]:
usersubsetgroup.get_group(1130)

,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104363,1130,1274,4.5
104443,1130,1968,4.5


In [37]:
usersubsetgroup=sorted(usersubsetgroup,key=lambda x:len(x[1]),reverse=True)
usersubsetgroup[:1],type(usersubsetgroup)


([(75,
         userId  movieId  rating
   7507      75        1     5.0
   7508      75        2     3.5
   7540      75      296     5.0
   7633      75     1274     4.5
   7673      75     1968     5.0)],
 list)

taking only top 100

In [38]:
usersubsetgroup=usersubsetgroup[0:100]


calculating pearson correlation and store it in a dictionary with key as userid and correlation as value 

In [39]:
from cmath import sqrt


pearsoncorrelation={}

for name,group in usersubsetgroup:
    group=group.sort_values(by='movieId')
    inputmovies=inputmovies.sort_values(by='movieId')

    n=len(group)

    temp_df=inputmovies[inputmovies['movieId'].isin(group['movieId'].tolist())]

    temp_ratinglist=temp_df['rating'].tolist()

    temp_grouplist=group['rating'].tolist()


    sxx=sum([i**2 for i in temp_ratinglist])-pow(sum(temp_ratinglist),2)/float(n)
    syy=sum([i**2 for i in temp_grouplist])-pow(sum(temp_grouplist),2)/float(n)
    sxy=sum([i*j for i,j in zip(temp_ratinglist,temp_grouplist)])-sum(temp_ratinglist)*sum(temp_grouplist)/float(n)


    if sxx!=0 and syy!=0:
        pearsoncorrelation[name]=sxy/sqrt(sxx*syy)

    else:
        pearsoncorrelation[name]=0

        
    

In [40]:
len(pearsoncorrelation),type(pearsoncorrelation)

(100, dict)

creating a dataframe from pearsoncorrelation dictionary

In [41]:
pearsoncorrelation_df=p.DataFrame.from_dict(pearsoncorrelation,orient='index')

pearsoncorrelation_df.head()

,0
75,0.827278+0.000000j
106,0.586009+0.000000j
686,0.832050+0.000000j
815,0.576557+0.000000j
1040,0.943456+0.000000j


In [42]:
pearsoncorrelation_df.columns=['similarityindex']
pearsoncorrelation_df['userId']=pearsoncorrelation_df.index
pearsoncorrelation_df.index=range(len(pearsoncorrelation_df))
pearsoncorrelation_df.head()

,similarityindex,userId
0,0.827278+0.000000j,75
1,0.586009+0.000000j,106
2,0.832050+0.000000j,686
3,0.576557+0.000000j,815
4,0.943456+0.000000j,1040


In [43]:
pearsoncorrelation_df['similarityindex']=pearsoncorrelation_df['similarityindex'].astype('float')
pearsoncorrelation_df.head()

c:\Users\hemas\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\cast.py:1181: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


,similarityindex,userId
0,0.827278,75
1,0.586009,106
2,0.832050,686
3,0.576557,815
4,0.943456,1040


getting top similar users

In [44]:
topusers=pearsoncorrelation_df.sort_values(by='similarityindex',ascending=False)[0:50]
topusers[0:10]

,similarityindex,userId
64,0.961678,12325
34,0.961538,6207
55,0.961538,10707
67,0.960769,13053
4,0.943456,1040
59,0.937614,11769
62,0.929294,12120
80,0.903584,15157
70,0.895144,13366
17,0.895144,3040


In [45]:
rating_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


merging topusers and ratings. to get movies of similar users 

In [52]:
topuserrating=topusers.merge(rating_df)
topuserrating.head()

,similarityindex,userId,movieId,rating
0,0.961678,12325,1,3.5
1,0.961678,12325,2,1.5
2,0.961678,12325,3,3.0
3,0.961678,12325,5,0.5
4,0.961678,12325,6,2.5


multiplying the similarityindex and rating

In [58]:
topuserrating['weightrating']=topuserrating['similarityindex']*topuserrating['rating']
topuserrating.head()

,similarityindex,userId,movieId,rating,weightrating
0,0.961678,12325,1,3.5,3.365874
1,0.961678,12325,2,1.5,1.442517
2,0.961678,12325,3,3.0,2.885035
3,0.961678,12325,5,0.5,0.480839
4,0.961678,12325,6,2.5,2.404196


In [59]:
topmovies=topuserrating.groupby('movieId').sum()
topmovies=topmovies[['similarityindex','weightrating']]
topmovies.head()

,similarityindex,weightrating
movieId,,
1,38.376281,140.800834
2,38.376281,96.656745
3,10.253981,27.254477
4,0.929294,2.787882
5,11.723262,27.151751


In [49]:
recommend_df=p.DataFrame()
recommend_df['expected user rating']=topmovies['weightrating']/topmovies['similarityindex']
recommend_df['movieId']=topmovies.index
recommend_df.head(10)

,expected user rating,movieId
movieId,,
1,3.668955,1
2,2.518658,2
3,2.657941,3
4,3.000000,4
5,2.316058,5
6,3.753763,6
7,2.852687,7
8,3.068622,8
9,2.102158,9


In [50]:
recommend_df=recommend_df.sort_values(by='expected user rating',ascending=False)
recommend_df.head(10)

,expected user rating,movieId
movieId,,
5073,5.0,5073
3329,5.0,3329
2284,5.0,2284
26801,5.0,26801
6776,5.0,6776
6672,5.0,6672
3759,5.0,3759
3769,5.0,3769
3775,5.0,3775


getting movies from movie_df 

In [51]:
movies_df[movies_df['movieId'].isin(recommend_df.head(15)['movieId'].tolist())]

,movieId,title,year
119,121,"Boys of St. Vincent, The",1992
2200,2284,Bandit Queen,1994
3243,3329,"Year My Voice Broke, The",1987
3449,3539,"Filth and the Fury, The",2000
3669,3759,Fun and Fancy Free,1947
3679,3769,Thunderbolt and Lightfoot,1974
3685,3775,Make Mine Music,1946
3686,3776,Melody Time,1948
3759,3851,I'm the One That I Want,2000
4978,5073,"Son's Room, The (Stanza del figlio, La)",2001
